### Installation
Install the packages required for executing this notebook.

## Some of the source codes are based on
https://towardsdatascience.com/how-to-set-up-custom-vertex-ai-pipelines-step-by-step-467487f81cad 

In [14]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [15]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed. The KFP SDK version should be >=1.6.

In [2]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.3.0
google-cloud-aiplatform==1.34.0
google_cloud_pipeline_components version: 2.4.1


In [3]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,ModelDeployOp)
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [18]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "assignment-1-399115"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2023_2056332"
# Specify your assignment
ASSIGNMENT = "Assignment_1"

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 
Refer to  https://www.kubeflow.org/docs/components/pipelines/v2/components/lightweight-python-components/ for more information.

#### Pipeline Component : Download and Merge

In [ ]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, feature_dataset_name: str, label_dataset_name: str, merged_dataset: Output[Dataset]):
    '''Download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Get the client and bucket
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)

     # Download the feature dataset
    blob1 = bucket.blob(feature_dataset_name)
    blob1.download_to_filename(feature_dataset_name.path + ".csv")
    df_feature = pd.read_csv(feature_dataset_name.path + ".csv")
    
    
    # Download the label dataset
    blob2 = bucket.blob(label_dataset_name)
    blob2.download_to_filename(label_dataset_name.path + ".csv")
    df_label = pd.read_csv(feature_dataset_name.path + ".csv")
    
    
    # Merge the datasets
    merged_df = pd.merge(df_feature, df_label, on='Ind_ID')
    merged_df.to_csv(merged_dataset.path + ".csv", index=False, encoding='utf-8-sig')
    

    logging.info('Download & Merge all Data complete!')

### Clean the dataset

In [ ]:
@dsl.component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.10.7-slim"
)
def clean_data(merged_dataset: Input[Dataset], cleaned_dataset: Output[Dataset]):
    '''Deletes irrelevant columns and removes NA's'''
    import pandas as pd
    import logging
    import sys
    import numpy

    # Sets the logging config
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 

    #Loads the merged dataset in
    df = pd.read_csv(merged_dataset.path, index_col=None)

    # Drops the columns: Birthday_count, Employed_days, Mobile_phone, Work_Phone, Phone, EMAIL_ID, Type_Occupation and Family_Members.
    cleandf = df.drop(columns= ["Birthday_count", "Employed_days", "Mobile_phone", "Work_Phone", "Phone", "EMAIL_ID", "Type_Occupation", "Family_Members"])

    # Save the cleaned dataset
    cleandf.to_csv(cleaned_dataset.path + ".csv", index=False, encoding='utf-8-sig')
    
    


### Split the dataset With Train-Test-Split

In [5]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def train_test_split(cleaned_dataset: Input[Dataset], dataset_train: Output[Dataset], dataset_test: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split as tts
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    alldata = pd.read_csv(cleaned_dataset.path, index_col=None)
    train, test = tts(alldata, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Training LogisticRegression

In [7]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def train_lr(train_dataset: Input[Dataset], model: Output[Model], metrics_lr: Output[Metrics]):
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression        
    import pickle 
    
    # Read the training dataset
    df = pd.read_csv(train_dataset.path + ".csv")

    # Splitting the label and feature data
    y_train = df["label"]
    X_train = df.drop("label", axis = 1)

    # Initiating a logistic regression model with default parameters
    model_lr = LogisticRegression()
    model_lr.fit(X_train, y_train)

    # Get the F1 Score score of the lr model
    metrics_lr.f1_score(X_train,y_train, average="binary")

    # Save metadata to the Logistic regression model
    model.metadata["framework"] = "LR"
    model.metadata["parameters"] = "Default"
    model.metadata["f1_score"] = float(metrics_lr)



    # Save the model as pickle file 
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_lr, file)   


#### Pipeline Component : Training KNeighborsClassifier

In [ ]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def train_knn(train_dataset: Input[Dataset], model: Output[Model], metrics_knn: Output[Metrics]):
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier        
    import pickle 
    
    # Read the training dataset
    df = pd.read_csv(train_dataset.path + ".csv")

    # Splitting the label and feature data
    y_train = df["label"]
    X_train = df.drop("label", axis = 1)
    
    # Initiate a KNN with default parameters
    model_knn = KNeighborsClassifier()
    model_knn.fit(X_train, y_train)

    # Get the F1 Score score of both models
    metrics_knn.f1_score(X_train,y_train, average="binary")

    # Save metadata to the KNN model
    model.metadata["framework"] = "KNN"
    model.metadata["parameters"] = "Default"
    model.metadata["f1_score"] = float(metrics_knn) 

    # Save the model as pickle file 
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_knn, file)   

#### Pipeline Component : Algorithm selection

In [ ]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(metrics_knn: dict, metrics_lr: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(metrics_knn)
    logging.info(metrics_lr)
    if metrics_knn.get("f1_score") > metrics_lr.get("f1_score"):
        return "KNN"
    else :
        return "LR"

#### Pipeline Component: Logistic regression prediction

In [ ]:
@dsl.component(
    packages_to_install = [
       "pandas", "scikit-learn", "numpy"
    ], base_image="python:3.10.7-slim"
)
def predict_lr(model_lr: Input[Model],
               test_data: Input[Dataset],
               results: Output[Dataset]):
    # Load in packages
    import pandas as pd
    import json
    import logging
    import sys
    import os
    import pickle

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    # Load the test data
    df = pd.read_csv(test_data.path+".csv")
    
    # Loading the saved logistic regression model with joblib
    m_filename = model_lr.path + ".pkl"
    lr_model = pickle.load(open(m_filename, 'rb'))

    # Split the test and train data
    X_test = df.drop(columns=["class"])
    y_test = df['class']

    # Get and log the predictions of the knn model
    y_pred = lr_model.predict(y_test)
    logging.info(y_pred)

    #Loading the saved model
    xNew = df[['ntp', 'pgc', 'dbp', 'tsft', 'si', 'bmi', 'dpf', 'age']]

    dfcp = df.copy()   
    y_classes = model_lr.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')
    

    

    

#### Pipeline Component: KNN Evaluation

In [ ]:
@dsl.component(
    packages_to_install = [
       "pandas", "scikit-learn", "numpy"
    ], base_image="python:3.10.7-slim"
)
def predict_knn(model_knn: Input[Model],
               test_data: Input[Dataset],
               results: Output[Dataset]):
    # Load in packages
    import pandas as pd
    import json
    import logging
    import sys
    import os
    import pickle

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    # Load the test data
    df = pd.read_csv(test_data.path+".csv")
    
    # Loading the saved knn model with joblib
    m_filename = model_knn.path + ".pkl"
    knn_model = pickle.load(open(m_filename, 'rb'))

    # Split the test and train data
    X_test = df.drop(columns=["class"])
    y_test = df['class']

    # Get and log the predictions of the knn model
    y_pred = knn_model.predict(y_test)
    logging.info(y_pred)
   
    #Loading the saved model
    xNew = df[['ntp', 'pgc', 'dbp', 'tsft', 'si', 'bmi', 'dpf', 'age']]

    dfcp = df.copy()   
    y_classes = knn_model.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')  
    


### Upload Model and Metrics to Google Bucket 

In [11]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, assignment: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob('model_{}.pkl'.format(assignment))
    source_file_name= model.path + '.pkl'
   
    blob.upload_from_filename(source_file_name)    
    
    print(f"File {source_file_name} uploaded to {model_repo}.")

### Deploy the model at Vertext AI 
We can use Google Pre-built Kebeflow componets such as  EndpointCreateOp, ModelUploadOp, and ModelDeployOp to deploy the models locally at Vertex AI.

***This is only for testing.  In your assigment, please use custom serving applications and CI-CD pipelines to deploy models. We should be able to deploy a given model at any given production environment. CI-CD pipelines are the best solution. ***

<img src="imgs/EnterpriseMlOps_Model_Deployment.png">

source: https://cloud.redhat.com/blog/enterprise-mlops-reference-design

https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction

https://cloud.google.com/vertex-ai/docs/pipelines/gcpc-list

#### Define the Pipeline

In [12]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="diabetes-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, dataset_uri: str, model_repo: str, thresholds_dict_str:str, model_repo_uri:str, assignment: str, dataset_feature_name: str, dataset_label_name: str):    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        feature_dataset_name= dataset_feature_name,
        label_dataset_name = dataset_label_name
    )
     
    clean_op = clean_data(merged_dataset =  di_op.outputs["merged_dataset"]) 
                                            # cleaned_dataset: Output[Dataset]) 
                                            
    train_test_split_op = train_test_split(cleaned_dataset = clean_op.outputs["cleaned_dataset"])
                                           # dataset_train: Output[Dataset], dataset_test: Output[Dataset])
        
    train_lr_op = train_lr(train_dataset = train_test_split_op.outputs["dataset_train"])
                                            # model: Output[Model])
    
    train_knn_op = train_knn(train_dataset = train_test_split_op.outputs["dataset_train"])
                                            # model: Output[Model]):
    
    predict_lr_op = predict_lr(model_lr = train_lr_op.outputs["model"],test_data = train_test_split_op.outputs["dataset_test"])
                                            # metrics_lr: Output[Metrics])

    predict_knn_op = predict_knn(model_knn = train_knn_op.outputs["model"],test_data = train_test_split_op.outputs["dataset_test"])
                                            # metrics_knn: Output[Metrics])

    algo_selection_op = compare_model(metrics_knn = evaluate_knn_op.outputs["metrics_knn"], metrics_lr = evaluate_lr_op.outputs["metrics_lr"]).after(evaluate_lr_op, evaluate_knn_op) 

    # defining the different conditions of algo selection                                 
    with dsl.If(algo_selection_op.output=="KNN"):
        evaluate_knn_op = predict_mlp(
            model=training_mlp_job_run_op.outputs["out_model"],      
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_mlp_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_mlp_job_run_op.outputs['out_model']
        ).after(predict_mlp_job_run_op)    
        
        import_unmanaged_model_task = dsl.importer(
            artifact_uri=model_repo_uri,
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                "containerSpec": {
                    "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",  # see https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers  
                },
            },
        ).after(upload_model_to_gc_op)      
       
        # using Google's custom components for for uloading and deploying the model.
       
        model_upload_op = ModelUploadOp(
            project=project_id,
            display_name="diabetes-prediction-model",
            unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
        ).after(import_unmanaged_model_task)       
               
        create_endpoint_op = EndpointCreateOp(
            project=project_id,
            display_name="diabetes-prediction-service",
        ).after(model_upload_op)      
        
        model_deploy_op = ModelDeployOp(
            model=model_upload_op.outputs["model"],
            endpoint=create_endpoint_op.outputs['endpoint'],
            deployed_model_display_name="diabetes-prediction-model",
            dedicated_resources_machine_type="n1-standard-4",
            dedicated_resources_min_replica_count=1,
            dedicated_resources_max_replica_count=1,
            traffic_split={"0": 100},
        ).after(create_endpoint_op)      

#### Compile the pipeline into a JSON file

In [13]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='diabetes_prdictor_training_pipeline.yaml')

#### Submit the pipeline run

In [14]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="diabetes-predictor",
    enable_caching=False,
    template_path="diabetes_prdictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2023',  # makesure to use your data bucket name 
        'dataset_uri':'gs://data_de2023/training_set.csv',
        'model_repo':'models_de2023', # makesure to use your model bucket name 
        'model_repo_uri':'gs://models_de2023',
        'assignment' : ASSIGNMENT # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/448730450443/locations/us-central1/pipelineJobs/diabetes-predictor-training-pipeline-20231004145834
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/448730450443/locations/us-central1/pipelineJobs/diabetes-predictor-training-pipeline-20231004145834')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/diabetes-predictor-training-pipeline-20231004145834?project=448730450443
PipelineJob projects/448730450443/locations/us-central1/pipelineJobs/diabetes-predictor-training-pipeline-20231004145834 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/448730450443/locations/us-central1/pipelineJobs/diabetes-predictor-training-pipeline-20231004145834 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/448730450443/locations/us-central1/pipelineJobs/diabetes-predictor-training-pipeline-202310041458